In [13]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [14]:
df = pd.read_csv(Path('./data/20211031.csv').resolve())
df

,Timestamp,What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,24/10/2021 13:57:51,OOPS! ALL CRONENBERG,HALLOWEEN OBVIOUSLY,HORROR-COMEDY,LONERS,[Default to Trish Pic],OOPS! ALL ZACK BRAFF
1,24/10/2021 13:58:05,HALLOWEEN OBVIOUSLY,HORROR-COMEDY,LONERS,OOPS! ALL CRONENBERG,[Default to Trish Pic],OOPS! ALL ZACK BRAFF
2,24/10/2021 14:29:14,OOPS! ALL CRONENBERG,LONERS,HORROR-COMEDY,[Default to Trish Pic],HALLOWEEN OBVIOUSLY,OOPS! ALL ZACK BRAFF
3,24/10/2021 14:57:02,HALLOWEEN OBVIOUSLY,HORROR-COMEDY,OOPS! ALL ZACK BRAFF,OOPS! ALL CRONENBERG,LONERS,[Default to Trish Pic]
4,24/10/2021 16:32:20,OOPS! ALL ZACK BRAFF,LONERS,NaN,NaN,NaN,NaN
5,24/10/2021 16:40:30,HALLOWEEN OBVIOUSLY,HORROR-COMEDY,[Default to Trish Pic],NaN,NaN,NaN
6,24/10/2021 17:18:18,HORROR-COMEDY,HALLOWEEN OBVIOUSLY,LONERS,OOPS! ALL CRONENBERG,[Default to Trish Pic],NaN


In [15]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [16]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array([], dtype=float64)

In [17]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [18]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [19]:
submission_dict

{}

In [20]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [21]:
ballots

[<Ballot()>,
 <Ballot()>,
 <Ballot()>,
 <Ballot()>,
 <Ballot()>,
 <Ballot()>,
 <Ballot()>]

In [22]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [23]:
print(election_result)

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
Blank votes        7  Rejected



In [24]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                 Votes  Status
----------------------  -------  --------
HALLOWEEN OBVIOUSLY           3  Hopeful
OOPS! ALL CRONENBERG          2  Hopeful
HORROR-COMEDY                 1  Rejected
OOPS! ALL ZACK BRAFF          1  Rejected
LONERS                        0  Rejected
[Default to Trish Pic]        0  Rejected

FINAL RESULT
Candidate                 Votes  Status
----------------------  -------  --------
HALLOWEEN OBVIOUSLY           4  Elected
OOPS! ALL CRONENBERG          2  Rejected
HORROR-COMEDY                 0  Rejected
OOPS! ALL ZACK BRAFF          0  Rejected
LONERS                        0  Rejected
[Default to Trish Pic]        0  Rejected
Blank votes                   1  Rejected

